In [ ]:
import json
import os
from typing import Any, List, Optional, Dict

import boto3
import pandas as pd
import streamlit as st
from langchain.llms.base import LLM
from langchain_experimental.agents import create_csv_agent
from pydantic import BaseModel, Field


class BedrockLLMConfig(BaseModel):
    client: Any = Field(..., description="The boto3 client for Bedrock")
    model: str = Field(..., description="The model name")

    class Config:
        arbitrary_types_allowed = True


class BedrockLLM(LLM):
    config: BedrockLLMConfig

    def __init__(self, client: Any, model: str):
        config = BedrockLLMConfig(client=client, model=model)
        super().__init__(config=config)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        body = json.dumps({
            "prompt": f"<s>[INST] {prompt} [/INST]",
            "max_tokens": 4096,
            "temperature": 0.5,
            "top_p": 0.9,
        })
        try:
            response = self.config.client.invoke_model(
                body=body,
                modelId=self.config.model,
                contentType="application/json",
                accept="application/json",
            )
            response_body = json.loads(response['body'].read())
            return response_body.get('outputs', [{}])[0].get('text', '')
        except Exception as e:
            print(f"Error calling Bedrock: {str(e)}")
            return f"Error: {str(e)}"

    @property
    def _llm_type(self) -> str:
        return "custom_bedrock"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {"model": self.config.model}


def save_uploaded_file(uploaded_file):
    temp_directory = "temp_files"
    os.makedirs(temp_directory, exist_ok=True)
    file_path = os.path.join(temp_directory, uploaded_file.name)
    with open(file_path, "wb") as f:
        f.write(uploaded_file.getbuffer())
    return file_path


def format_output(output: str) -> str:
    # Clean and format the output
    lines = output.split('\n')
    final_answer = None
    actions = []

    for line in lines:
        if line.startswith("Final Answer:"):
            final_answer = line
        elif line.startswith("Observation:"):
            observation = line.split("Observation:", 1)[1].strip()
            if observation.startswith("City"):
                observation = "\n" + observation.replace("dtype: float64", "")
                actions.append(observation)

    formatted_output = "\n\n".join(actions)
    if final_answer:
        formatted_output += "\n\n" + final_answer

    return formatted_output if formatted_output else output


def main():
    # Set AWS credentials (replace placeholders with your actual credentials)
    aws_access_key = ''
    aws_secret_key = ''
    aws_region = 'us-east-1'

    # Set AWS environment variables
    os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key
    os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_key
    os.environ['AWS_DEFAULT_REGION'] = aws_region

    # AWS model configuration
    aws_model_name = 'mistral.mixtral-8x7b-instruct-v0:1'

    st.set_page_config(page_title="CSV Consultant")
    st.header("Ask your CSV 📈")

    csv_file = st.file_uploader("Upload a CSV file", type="csv")
    if csv_file is not None:
        file_path = save_uploaded_file(csv_file)

        try:
            # Initialize Bedrock client
            bedrock_client = boto3.client(
                service_name="bedrock-runtime",
                region_name=aws_region,
            )

            # Initialize the custom LLM
            llm = BedrockLLM(client=bedrock_client, model=aws_model_name)

            # Create LangChain agent for CSV interaction
            agent = create_csv_agent(
                llm=llm,
                path=file_path,
                verbose=True,
                handle_parsing_errors=True,  # Enable handling of parsing errors
                allow_python_repl=True,
                allow_dangerous_code=True  # Be cautious with this setting
            )

            user_question = st.text_input("Ask a question about your CSV: ")

            if user_question and user_question.strip():
                with st.spinner(text="In progress..."):
                    try:
                        response = agent.run(user_question)
                        formatted_response = format_output(response)
                        st.write(formatted_response)
                    except Exception as e:
                        st.error(f"An error occurred while running the agent: {str(e)}")

        except Exception as e:
            st.error(f"An error occurred during setup: {str(e)}")


if __name__ == "__main__":
    main()
